Importing modules

In [18]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

Importing datasets

In [19]:
mnistData,mnistInfo = tfds.load(name='mnist',with_info=True,as_supervised=True)

In [21]:
mnistTrain = mnistData['train']
mnistTest = mnistData['test']

In [37]:
numValidationSamples = 0.1 * mnistInfo.splits['train'].num_examples
numValidationSamples = tf.cast(numValidationSamples,tf.int64)

numTrainSamples = tf.cast(mnistInfo.splits['train'].num_examples,tf.int64) 
numTestSamples = tf.cast(mnistInfo.splits['test'].num_examples,tf.int64) 

In [38]:
#Function to scale image data

In [44]:
def scale(image,label):
    image = tf.cast(image,tf.float64)
    image /= 255.0
    return image,label

In [62]:
#Scaling the data 
trainSample = mnistTrain.map(scale)
testSample = mnistTest.map(scale)

In [63]:
BUFFER_SIZE = 10000
#Randamizing the data
trainSample = trainSample.shuffle(BUFFER_SIZE)

#Taking 10% of the data for validation
validationData = trainSample.take(numValidationSamples)

#Taking remaining data for training
trainData = trainSample.skip(numValidationSamples)

#Batching the data
BATCH_SIZE = 100
trainData = trainData.batch(BATCH_SIZE)
testData = testSample.batch(numTestSamples)
validationData = validationData.batch(numValidationSamples)

validationInputs,validationTargets = next(iter(validationData))

In [58]:
#Configuring the model

In [60]:
inputSize = 784
hiddenLayerSize = 50
outputSize = 10

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hiddenLayerSize,activation='relu'),
                            tf.keras.layers.Dense(hiddenLayerSize,activation='relu'),
                            tf.keras.layers.Dense(outputSize,activation='softmax')
                            ])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])


In [61]:
#Training the model
NUM_EPOCHS = 5
model.fit(trainData,epochs=NUM_EPOCHS,validation_data=(validationInputs,validationTargets),verbose=2)

Epoch 1/5
540/540 - 8s - loss: 0.4125 - accuracy: 0.8810 - val_loss: 0.2195 - val_accuracy: 0.9375 - 8s/epoch - 15ms/step
Epoch 2/5
540/540 - 5s - loss: 0.1736 - accuracy: 0.9493 - val_loss: 0.1519 - val_accuracy: 0.9567 - 5s/epoch - 9ms/step
Epoch 3/5
540/540 - 5s - loss: 0.1292 - accuracy: 0.9616 - val_loss: 0.1173 - val_accuracy: 0.9660 - 5s/epoch - 10ms/step
Epoch 4/5
540/540 - 5s - loss: 0.1071 - accuracy: 0.9682 - val_loss: 0.1060 - val_accuracy: 0.9662 - 5s/epoch - 10ms/step
Epoch 5/5
540/540 - 5s - loss: 0.0935 - accuracy: 0.9714 - val_loss: 0.0903 - val_accuracy: 0.9743 - 5s/epoch - 10ms/step


In [64]:
#Testing the model
testLoss , accuracy = model.evaluate(testData)

1/1 [==============================] - 1s 921ms/step - loss: 0.1039 - accuracy: 0.9690


In [73]:
print('Test loss :{0:.2f}. Test accuray : {1:.2f}%'.format(testLoss,accuracy*100))

Test loss :0.10. Test accuray : 96.90%
